### SQL Code in Microsoft SQL Server to Create View for Dataset

Create View HumanResources.vEmployeeHRProject AS
(
	SELECT 
	edh.BusinessEntityID,
	edh.DepartmentID,
	edh.ShiftID,
	edh.StartDate, 
	edh.EndDate,
	e.JobTitle,
	e.BirthDate,
	e.MaritalStatus,
	e.Gender,
	e.HireDate,
	e.SalariedFlag,
	e.VacationHours,
	e.SickLeaveHours,
	e.OrganizationLevel,
	p.PersonType,
	p.FirstName,
	p.LastName,
	d.Name DepartmentName, 
	d.GroupName DepartmentGroup,
	eph.RateChangeDate,
	eph.Rate,
	eph.PayFrequency,
	s.Name ShiftName,
	s.StartTime ShiftStart,
	s.EndTime ShiftEnd,
	bea.AddressID,
	a.City,
	a.PostalCode,
	pp.PhoneNumber,
	pp.PhoneNumberTypeID,
	pnt.Name PhoneType

	FROM HumanResources.EmployeeDepartmentHistory edh
	LEFT JOIN HumanResources.Employee e
	ON e.BusinessEntityID = edh.BusinessEntityID
	LEFT JOIN Person.Person p
	ON p.BusinessEntityID = edh.BusinessEntityID
	LEFT JOIN HumanResources.Department d
	ON d.DepartmentID = edh.DepartmentID
	LEFT JOIN HumanResources.EmployeePayHistory eph
	ON eph.BusinessEntityID = edh.BusinessEntityID
	LEFT JOIN HumanResources.Shift s
	ON s.ShiftID = edh.ShiftID
	LEFT JOIN Person.BusinessEntityAddress bea
	ON bea.BusinessEntityID = edh.BusinessEntityID
	LEFT JOIN Person.Address a
	ON a.AddressID = bea.AddressID
	LEFT JOIN Person.PersonPhone pp
	ON pp.BusinessEntityID = edh.BusinessEntityID
	LEFT JOIN Person.PhoneNumberType pnt
	ON pnt.PhoneNumberTypeID = pp.PhoneNumberTypeID
)

### Load Libraries and Dataset via SQL

In [1]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from datetime import datetime 
from matplotlib.ticker import FuncFormatter

# Define the connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost\\SQLEXPRESS;'
    'DATABASE=AdventureWorks2022;'
    'Trusted_Connection=yes;'
)


# Write the SQL query to select data from the view
sql_query = "SELECT * FROM HumanResources.vEmployeeHRProject ORDER BY BusinessEntityID"

# Load the data from SQL Server into a pandas DataFrame
df = pd.read_sql(sql_query, conn)

# Display the first few rows of the DataFrame
df.head()

# Close the connection
conn.close()

C:\Users\rbrod\AppData\Local\Temp\ipykernel_31636\2139792467.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


### Clean Data

In [2]:
# Step 1: Filter for rows where 'EndDate' is null
filtered_df = df[df['EndDate'].isnull()]

# Step 2: Sort by 'RateChangeDate' within each 'BusinessEntityID'
# and keep only the latest one
cleaned_df = (filtered_df
              .sort_values('RateChangeDate', ascending=False)
              .groupby('BusinessEntityID')
              .first()  # Keep the first entry after sorting (latest RateChangeDate)
              .reset_index())

# Fill NaN values in 'OrganizationLevel' with 0
cleaned_df['OrganizationLevel'] = cleaned_df['OrganizationLevel'].fillna(0)

In [3]:
cleaned_df

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,...,PayFrequency,ShiftName,ShiftStart,ShiftEnd,AddressID,City,PostalCode,PhoneNumber,PhoneNumberTypeID,PhoneType
0,1,16,1,2009-01-14,None,Chief Executive Officer,1969-01-29,S,M,2009-01-14,...,2,Day,07:00:00,15:00:00,249,Newport Hills,98006,697-555-0142,1,Cell
1,2,1,1,2008-01-31,None,Vice President of Engineering,1971-08-01,S,F,2008-01-31,...,2,Day,07:00:00,15:00:00,293,Renton,98055,819-555-0175,3,Work
2,3,1,1,2007-11-11,None,Engineering Manager,1974-11-12,M,M,2007-11-11,...,2,Day,07:00:00,15:00:00,224,Redmond,98052,212-555-0187,1,Cell
3,4,2,1,2010-05-31,None,Senior Tool Designer,1974-12-23,S,M,2007-12-05,...,2,Day,07:00:00,15:00:00,11387,Minneapolis,55402,612-555-0100,1,Cell
4,5,1,1,2008-01-06,None,Design Engineer,1952-09-27,M,F,2008-01-06,...,2,Day,07:00:00,15:00:00,190,Bellevue,98004,849-555-0139,1,Cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,286,3,1,2013-05-30,None,Sales Representative,1977-02-14,S,F,2013-05-30,...,2,Day,07:00:00,15:00:00,36,Melbourne,3000,1 (11) 500 555-0190,1,Cell
286,287,3,1,2012-04-16,None,European Sales Manager,1957-09-20,M,F,2012-04-16,...,2,Day,07:00:00,15:00:00,300,Renton,98055,775-555-0164,3,Work
287,288,3,1,2013-05-30,None,Sales Representative,1975-07-09,S,F,2013-05-30,...,2,Day,07:00:00,15:00:00,35,Berlin,14111,1 (11) 500 555-0140,1,Cell
288,289,3,1,2012-05-30,None,Sales Representative,1968-03-17,M,F,2012-05-30,...,2,Day,07:00:00,15:00:00,37,Cambridge,BA5 3HX,1 (11) 500 555-0145,3,Work


### Add Age and Salary Columns for Chart

In [4]:
# Convert 'BirthDate' to datetime
cleaned_df['BirthDate'] = pd.to_datetime(cleaned_df['BirthDate'])

# Ensure 'Rate' is a string before removing the dollar sign and converting to numeric
cleaned_df['Rate'] = cleaned_df['Rate'].astype(str).str.replace('$', '').str.strip()
cleaned_df['Rate'] = pd.to_numeric(cleaned_df['Rate'], errors='coerce')

# Calculate Age
current_date = datetime.now()
cleaned_df['Age'] = (current_date - cleaned_df['BirthDate']).dt.days // 365

# Calculate Salary based on hourly rate (assuming 2080 working hours in a year)
cleaned_df['Salary'] = cleaned_df['Rate'] * 2080  # Annual salary

### Create Dashboard Draft

In [5]:
import panel as pn
import matplotlib.pyplot as plt

# Ensure Panel extension is loaded
pn.extension()

# Function to update the dashboard based on department selection
def update_dashboard(department):
    if department == 'All Departments':
        df_filtered = cleaned_df
    else:
        df_filtered = cleaned_df[cleaned_df['DepartmentName'] == department]
    
    # Total Employees
    total_employees = len(df_filtered)
    
    # Marital Status Donut Plot
    marital_status_counts = df_filtered['MaritalStatus'].value_counts()
    marital_status_fig, ax = plt.subplots()
    ax.pie(marital_status_counts, labels=marital_status_counts.index, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.3))
    ax.set_title('Marital Status Distribution')
    plt.close(marital_status_fig)  # Close the figure after creating it
    
    # Gender Donut Plot
    gender_counts = df_filtered['Gender'].value_counts()
    gender_fig, ax = plt.subplots()
    ax.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.3))
    ax.set_title('Gender Distribution')
    plt.close(gender_fig)  # Close the figure after creating it
    
    # Age vs Salary Scatter Plot
    scatter_fig, ax = plt.subplots()
    ax.scatter(df_filtered['Age'], df_filtered['Salary'])
    ax.set_xlabel('Age')
    ax.set_ylabel('Salary')
    ax.set_title('Age vs Salary')
    plt.close(scatter_fig)  # Close the figure after creating it
    
    # Organization Level Plot
    org_level_counts = df_filtered['OrganizationLevel'].value_counts().sort_index()
    org_level_fig, ax = plt.subplots()
    ax.bar(org_level_counts.index, org_level_counts.values)
    ax.set_xlabel('Organization Level')
    ax.set_ylabel('Count')
    ax.set_title('Organization Level Distribution')
    plt.close(org_level_fig)  # Close the figure after creating it
    
    # Create the dashboard layout
    dashboard = pn.Column(
        pn.Row(
            pn.Column(
                pn.pane.Markdown(f"## Total Employees: {total_employees}"),
                pn.pane.Matplotlib(marital_status_fig),
                pn.pane.Matplotlib(gender_fig)
            ),
            pn.Column(
                pn.pane.Matplotlib(scatter_fig),
                pn.pane.Matplotlib(org_level_fig)
            )
        )
    )
    
    return dashboard

# Department selector widget
department_selector = pn.widgets.Select(name='Department', options=['All Departments'] + cleaned_df['DepartmentName'].unique().tolist(), value='All Departments')

# Bind the update function to the department selector
dashboard = pn.bind(update_dashboard, department_selector)

# Display the dashboard
pn.Column(department_selector, dashboard).servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'33c744fb-f7fa-4fce-ba58-1fddab19b0fa': {'version…

### Alter Dashboard Layout

In [6]:
import panel as pn
import matplotlib.pyplot as plt

# Ensure Panel extension is loaded
pn.extension(sizing_mode="stretch_width")

# Function to update the dashboard based on department selection
def update_dashboard(department):
    if department == 'All Departments':
        df_filtered = cleaned_df
    else:
        df_filtered = cleaned_df[cleaned_df['DepartmentName'] == department]
    
    # Total Employees
    total_employees = len(df_filtered)
    
    # Marital Status Donut Plot
    marital_status_counts = df_filtered['MaritalStatus'].value_counts()
    marital_status_fig, ax = plt.subplots(figsize=(4, 4))
    ax.pie(marital_status_counts, labels=marital_status_counts.index, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.3))
    ax.set_title('Marital Status', fontsize=10)
    plt.close(marital_status_fig)
    
    # Gender Donut Plot
    gender_counts = df_filtered['Gender'].value_counts()
    gender_fig, ax = plt.subplots(figsize=(4, 4))
    ax.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90, wedgeprops=dict(width=0.3))
    ax.set_title('Gender Distribution', fontsize=10)
    plt.close(gender_fig)
    
    # Age vs Salary Scatter Plot
    scatter_fig, ax = plt.subplots(figsize=(5, 4))
    ax.scatter(df_filtered['Age'], df_filtered['Salary'], s=10)
    ax.set_xlabel('Age', fontsize=10)
    ax.set_ylabel('Salary', fontsize=10)
    ax.set_title('Age vs Salary', fontsize=10)
    plt.close(scatter_fig)
    
    # Organization Level Plot
    org_level_counts = df_filtered['OrganizationLevel'].value_counts().sort_index()
    org_level_fig, ax = plt.subplots(figsize=(5, 4))
    ax.bar(org_level_counts.index, org_level_counts.values)
    ax.set_xlabel('Organization Level', fontsize=10)
    ax.set_ylabel('Count', fontsize=10)
    ax.set_title('Org Level Distribution', fontsize=10)
    plt.close(org_level_fig)
    
    # Create the dashboard layout using GridSpec for alignment
    grid = pn.GridSpec(sizing_mode='stretch_both', max_height=800, max_width=1000)

    # Add widgets and plots to the grid
    grid[0, :2] = pn.pane.Matplotlib(marital_status_fig, width=300, height=300)
    grid[0, 2:4] = pn.pane.Matplotlib(gender_fig, width=300, height=300)
    grid[1, :2] = pn.pane.Matplotlib(scatter_fig, width=450, height=300)
    grid[1, 2:4] = pn.pane.Matplotlib(org_level_fig, width=450, height=300)
    
    # Add Total Employees as text at the top of the grid
    total_text = pn.pane.Markdown(f"### Total Employees: {total_employees}")
    
    return pn.Column(total_text, grid)

# Department selector widget
department_selector = pn.widgets.Select(name='Department', options=['All Departments'] + cleaned_df['DepartmentName'].unique().tolist(), value='All Departments')

# Bind the update function to the department selector
dashboard = pn.bind(update_dashboard, department_selector)

# Add a title for the dashboard using HTML for centering
title = pn.pane.Markdown("<h2 style='text-align: center;'>Adventure Works 2022 HR Dashboard</h2>")

# Display the dashboard with a fixed size of 1000x800
pn.Column(title, department_selector, dashboard, width=1000, height=800).servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'8ebb8d53-061c-433a-886e-05b902c8ad48': {'version…

### Create Final Dashboard, Finalize Details

In [7]:
import panel as pn
import matplotlib.pyplot as plt

# Ensure Panel extension is loaded
pn.extension(sizing_mode="stretch_width")

# Custom CSS for the dropdown
css = """
<style>
    .custom-dropdown {
        background-color: white;
        border: 1px solid #1cbe92;  /* Optional: Border color */
        color: #1cbe92;              /* Optional: Text color */
    }
</style>
"""
pn.config.raw_css.append(css)

# Function to update the dashboard based on department selection
def update_dashboard(department):
    if department == 'All Departments':
        df_filtered = cleaned_df
    else:
        df_filtered = cleaned_df[cleaned_df['DepartmentName'] == department]
    
    # Total Employees
    total_employees = f' in {department} is {len(df_filtered)}'

    # Marital Status Donut Plot
    marital_status_counts = df_filtered['MaritalStatus'].value_counts()
    marital_status_labels = ["Married (M)" if label == "M" else "Single (S)" for label in marital_status_counts.index]
    marital_status_fig, ax = plt.subplots(figsize=(5, 5))
    ax.pie(marital_status_counts, labels=marital_status_labels, autopct='%1.1f%%', startangle=45, wedgeprops=dict(width=0.25), colors=['#254b7f', '#CE7623'])
    ax.text(0, 0, 'Marital Status', fontsize=14, ha='center', va='center', fontweight='bold')
    plt.tight_layout()
    plt.close(marital_status_fig)
    
    # Gender Donut Plot
    gender_counts = df_filtered['Gender'].value_counts()
    gender_labels = ["Male (M)" if label == "M" else "Female (F)" for label in gender_counts.index]
    gender_fig, ax = plt.subplots(figsize=(5, 5))
    ax.pie(gender_counts, labels=gender_labels, autopct='%1.1f%%', startangle=0, wedgeprops=dict(width=0.25),colors=['#254b7f', '#CE7623'])
    ax.text(0, 0, 'Gender', fontsize=14, ha='center', va='center', fontweight='bold')
    plt.tight_layout()
    plt.close(gender_fig)

    # Pay Frequency Donut Plot
    pay_frequency_counts = df_filtered['PayFrequency'].map({1: "Weekly", 2: "Bi-Weekly"}).value_counts()
    pay_frequency_fig, ax = plt.subplots(figsize=(5, 5))
    ax.pie(pay_frequency_counts, labels=pay_frequency_counts.index, autopct='%1.1f%%', startangle=45, wedgeprops=dict(width=0.25),colors=['#254b7f', '#CE7623'])
    ax.text(0, 0, 'Pay Frequency', fontsize=14, ha='center', va='center', fontweight='bold')
    plt.tight_layout()
    plt.close(pay_frequency_fig)

    # Age vs Salary Scatter Plot
    scatter_fig, ax = plt.subplots(figsize=(9, 6))
    ax.scatter(df_filtered['Age'], df_filtered['Salary'], s=50, alpha=0.75, color='#254b7f')
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_title('Age vs Salary', fontsize=14, fontweight='bold')
    # Custom formatter to display y-axis labels as $10K, $20K, etc.
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'${int(x / 1000)}K'))
    sns.despine()
    plt.tight_layout()
    plt.close(scatter_fig)
    
   # Organization Level Plot
    org_level_counts = df_filtered['OrganizationLevel'].value_counts().sort_index()
    org_level_fig, ax = plt.subplots(figsize=(9, 6))
    ax.bar(org_level_counts.index, org_level_counts.values, color='#254b7f')
    ax.set_xlabel('Organization Level', fontsize=14)
    ax.set_ylabel('Number of Employees', fontsize=14)
    ax.set_title('Organization Level Distribution', fontsize=14, fontweight='bold')

    # Set x and y axes to show only whole numbers
    ax.xaxis.set_major_locator(mticker.MaxNLocator(integer=True))
    ax.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))
    sns.despine()
    plt.tight_layout()
    plt.close(org_level_fig)
    
    # Create the dashboard layout using GridSpec for alignment
    grid = pn.GridSpec(sizing_mode='stretch_both', max_height=1200, max_width=1400)
    grid.margin = (0, 0, 0, 0)
    grid.spacing = 0

    # Add widgets and plots to the grid
    grid[0, 8:12] = pn.pane.Matplotlib(marital_status_fig)
    grid[0, :4] = pn.pane.Matplotlib(gender_fig)
    grid[0, 4:8] = pn.pane.Matplotlib(pay_frequency_fig)
    grid[1, :6] = pn.pane.Matplotlib(scatter_fig)
    grid[1, 6:12] = pn.pane.Matplotlib(org_level_fig)
    
    # Add Total Employees as text at the top of the grid
    total_text = pn.pane.Markdown(f"### Total Employees {total_employees}")
    
    return pn.Column(total_text, grid)

# Department selector widget
department_selector = pn.widgets.Select(name='Department', options=['All Departments'] + cleaned_df['DepartmentName'].unique().tolist(), value='All Departments')

# Bind the update function to the department selector
dashboard = pn.bind(update_dashboard, department_selector)

# Add a title for the dashboard with custom color and background styling
title = pn.pane.Markdown(
    "<h2 style='text-align: center; color: #254b7f; background-color: white;'>Adventure Works 2022 HR Dashboard</h2>"
)

# Display the dashboard with a fixed size of 1400x800
pn.Column(title, department_selector, dashboard, width=1400, height=1100).servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'d47b1c48-ffe8-49cc-abe4-1a5dbb731d9d': {'version…